In [7]:
!pip install nltk
!pip install newspaper3k
!pip install numpy
!pip install networkx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=39105d62db03247e2999448f7853f9c6386bc4f79583e5fce9bc00e4bbb638f9
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=985dbdc966251d9677c8508ab180c9089f80eabd254e845c5381f1fcf355e97e
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

In [8]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import nltk
from newspaper import Article
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

def read_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(article_url, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read the article
    article_content = read_article(article_url)

    # Step 2 - Tokenize the article into sentences
    sentences = sent_tokenize(article_content)

    # Step 3 - Generate Sentence Similarity Matrix
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 4 - Rank sentences in Similarity Matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 5 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    # Step 6 - Pick top N sentences
    for i in range(top_n):
        summarize_text.append("".join(ranked_sentence[i][1]))

    return " ".join(summarize_text)

# Example usage
url = "https://timesofindia.indiatimes.com/city/bengaluru/bizman-loses-5-2cr-in-stock-app-fraud/articleshow/109483991.cms"
summary = generate_summary(url, top_n=3)
print(summary)

BENGALURU: A 52-year-old businessman dabbling in the stock market allegedly lost Rs 5.2 crore to cybercrooks recently.The victim, Sharath (name changed), director of a private firm and resident of Jayanagar, told cybercrime police in his complaint on April 8 that he received a WhatsApp message on March 11 about investing in the stock market to get high returns.The message also provided a link (bys-app.com). The miscreants gave him multiple accounts to transfer the money and claimed that it was invested in the stock market.By April 2, Sharath had ended up transferring Rs 5.2 crore to five accounts provided by the fraudsters. Sharath didn’t relent at first but the miscreants eventually managed to convince him to download the app.


BENGALURU: A 52-year-old businessman dabbling in the stock market allegedly lost Rs 5.2 crore to cybercrooks recently.The victim, Sharath (name changed), director of a private firm and resident of Jayanagar, told cybercrime police in his complaint on April 8 that he received a WhatsApp message on March 11 about investing in the stock market to get high returns.The message also provided a link (bys-app.com). The miscreants gave him multiple accounts to transfer the money and claimed that it was invested in the stock market.By April 2, Sharath had ended up transferring Rs 5.2 crore to five accounts provided by the fraudsters. Sharath didn’t relent at first but the miscreants eventually managed to convince him to download the app.